In [ ]:
# import built-in packages
from warnings import filterwarnings
import re

# import third-party packages
from geopy.geocoders import Nominatim
from pandas_profiling import ProfileReport
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import scipy as sp

In [ ]:
# filter out warnings to avoid unneccessary outputs
filterwarnings(
    'ignore'
)

In [ ]:
# set matplotlib to render plots within the notebook inline
%matplotlib inline

In [ ]:
# set custom pandas options
for option, value in {
    'display.max_columns': 50,
    'display.max_colwidth': None,
    'display.max_info_columns': 50,
    'display.max_rows': 20,
    'display.precision': 4,
    # 'plotting.backend': 'plotly'
}.items():
    pd.set_option(
        option,
        value        
    )

In [ ]:
# load each dataset into a pandas DataFrame object
cocoon_pharmacy_df = pd.read_csv(
    '../data/cocoon_center_pharmacy.csv'
)
data_literacy_df = pd.read_csv(
    '../data/data_literacy_questionnaire.csv'
)
data_journey_df = pd.read_csv(
    '../data/data_journey_questionnaire.csv'
)
meeting_cadence_df = pd.read_csv(
    '../data/meeting_cadence_survey.csv'
)

In [ ]:
# create an example function with one mandatory and one optional parameter
def display_with_info(
    dataframe: pd.DataFrame,
    name: str = None
) -> None:
    # print dataframe name if passed
    if name:
        print(
            '=' * len(name),
            end = '\n'
        )
        print(
            name,
            end = '\n' + (
                '=' * len(name)
            ) + '\n\n'
        )
    # display dataframe information
    display(
        dataframe.info()
    )
    # label the next output
    print(
        # create the illusion of a border by using hyphens
        '-' * 14
    )
    print(
        'First Ten Rows',
        # create the illusion of a border by using hyphens
        end = '\n' + (
            '-' * 14
        )
    )
    # display the first ten rows of the dataframe
    display(
        dataframe.head(
            10
        )
    )
    # label the next output
    print(
        # create the illusion of a border by using hyphens
        '\n',
        end = (
            '-' * 13
        ) + '\n'
    )
    print(
        'Last Ten Rows',
        # create the illusion of a border by using hyphens
        end = '\n' + (
            '-' * 13
        )
    )
    # display the last ten rows of the dataframe
    display(
        dataframe.tail(
            10
        )
    )

In [ ]:
# create dictionary of dataframes and their names
sample_datasets = {
    'Cocoon Center Pharmacy Items & Reviews': cocoon_pharmacy_df,
    'Data Literacy Questionnaire': data_literacy_df,
    'Data Journey Questionnaire': data_journey_df,
    'Meeting Cadence Survey': meeting_cadence_df
}
# iterate through (dataframe name - dataframe object) 'key - value' pairs 
for df_name, df_object in sample_datasets.items():
    # display each dataframe's summary information and first/last 10 rows
    display_with_info(
        df_object,
        df_name
    )
    # add a horizontal border to separate outputs
    print(
        '\n' + (
            '_' * 79
        ),
        end = '\n\n'
    )

In [ ]:
# iterate through sample datasets again but display profile report this time
for df_name, df_object in sample_datasets.items():
    display(
        ProfileReport(
            df_object,
            title = f'Profile of {df_name} DataFrame',
            html = {
                'navbar_show': True,
                'style': {
                    'primary_color': '#016ba9',
                    'full_width': True
                }
            },
            progress_bar = False
        ).to_notebook_iframe()
    )

#### ANALYZING CUSTOMER LOCATIONS

In [ ]:
# clean up the names of the customer locations
cocoon_pharmacy_df['customer_loc'] = cocoon_pharmacy_df['customer_loc'].str.strip(
).str.lower(
).replace(
    {
        'the kingdom':'saudi arabia',
        'good':None,
        'not available':None,
        'ksa':'saudi arabia',
        'the kingdom':'saudi arabia',
        'london o':'london, ontario',
        'cheshire':'cheshire, uk',
        '-':None,
        'sale, cheshire':'sale, cheshire, uk',
        'tilbury, essex':'tilbury, essex, uk',
        'middle east':'ksa middle east',
        'kingston':'kingston, uk',
        'awali':'awali, bahrain',
        'riyadh, s. a.':'riyadh, saudi arabia',
        'north':None,
        'south east':None,
        'memo':None
    }
)

In [ ]:
# initialize geolocator object to get coordinates from location names
geolocator = Nominatim(
    user_agent = "s"
)
location = geolocator.geocode("Welsh Border")
geolocator.geocode("Abu Dhabi")

In [ ]:
# create columns with coordinates for the customer locations
new_cocoon_df = cocoon_pharmacy_df.assign(
    Customer_Location_Latitude_Longitude = cocoon_pharmacy_df['customer_loc'].apply(
        # lamdda denotes an anonymous function where the 'caller' is the 'input'
        lambda x: geolocator.geocode(x) if pd.notnull(
            geolocator.geocode(x)
        ) else None
    ),
    # use the column created in the lines above to create the two columns below
    Latitude = cocoon_pharmacy_df['Customer_Location_Latitude_Longitude'].apply(
        lambda x: x.latitude if pd.notnull(x) else None
    ),
    Longitude = cocoon_pharmacy_df['Customer_Location_Latitude_Longitude'].apply(
        lambda x: x.longitude if pd.notnull(x) else None
    )
)

In [ ]:
# store the modified dataframe as a .csv file in the 'data' folder
new_cocoon_df.to_csv('../data/cocoon_pharmacy_location_added.csv')

In [ ]:
# overwrite the 'cocoon_pharmacy_df' object with a dataframe of our 'new' file
cocoon_pharmacy_df = pd.read_csv(
    '../data/cocoon_pharmacy_location_added.csv',
    index_col = 0
)

In [ ]:
# get list of columns in the dataframe excluding the ones pertaining to coordinates
columns_without_customer_location = set(
    cocoon_pharmacy_df.columns
).difference(
    [
        'Customer_Location_Latitude_Longitude',
        'Latitude',
        'Longitude'
    ]
)
# slice the dataframe for rows and columns without duplicates in the non-coordinate columns
cocoon_pharmacy_df = cocoon_pharmacy_df.loc[
    # slice the dataframe for columns without location and then drop the duplicates
    cocoon_pharmacy_df[columns_without_customer_location].drop_duplicates(
        # keep the first instance of a duplicate
        keep  = 'first'
    # use the index of this 'de-duped' dataframe to slice for the 'first' instance of each
    ).index,
    # denotes 'all' columns
    :
]

In [ ]:
# set default renderer in plotly to iframe and use online CDN
pio.renderers.default = "iframe_connected"

#### NUMBER OF REVIEWS PER BRAND

In [ ]:
plt.figure(figsize = (30,8))
ax = sns.countplot(data = cocoon_pharmacy_df, x = 'brand', color = 'blue')
for p in ax.patches:
    x = p.get_x()
    height = p.get_height()
    width = p.get_width()
    if pd.isnull(height):
        pass
    else:   
        ax.text(x+width/2, height, str(height) , ha = 'center', weight = 'bold', fontsize = 18)

plt.xticks(rotation = 45, fontsize = 18)
plt.yticks(fontsize = 18)
plt.xlabel('Brand', fontsize = 25)
plt.ylabel('Review Counts', fontsize = 25)
plt.title('Number of reviews per Brand - Total {}'.format(cocoon_pharmacy_df.shape[0]),size = 30)
plt.show()

#### NUMBER OF REVIEWS PER PRODUCT CATEGORY

In [ ]:
plt.figure(figsize = (30,8))
ax = sns.countplot(data = cocoon_pharmacy_df, x = 'product_cat', color = 'blue')
for p in ax.patches: 
    x = p.get_x()
    height = p.get_height()
    width = p.get_width()
    if pd.isnull(height):
        pass
    else:   
        ax.text(x+width/2, height, str(height) , ha = 'center', weight = 'bold', fontsize = 18)

plt.xticks(rotation = 45, fontsize = 18)
plt.yticks(fontsize = 18)
plt.xlabel('Product Category', fontsize = 25)
plt.ylabel('Review Counts', fontsize = 25)
plt.title('Number of reviews per Product Category', fontsize = 30)
plt.show()

#### NUMBER OF REVIEWS PER BRAND AND PRODUCT CATEGORY

In [ ]:
# create a new dataframe object to view where the 'form' is grouped by the 'product category' and 'brand'
cnt_df = cocoon_pharmacy_df.groupby(
    [
        'product_cat',
        'brand'
    ]
)['form'].count().reset_index().rename(
    {
        'form':'Review Count'
    },
    axis = 1
).reset_index()

In [ ]:

fig = px.bar(cnt_df, x = 'product_cat', y = 'Review Count', color= 'brand', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle=325, title = 'Brand')
fig.update_yaxes(title = 'Review Counts')
fig.update_layout(title = dict(
        text='<b>Number of reviews per Product Category and Brand</b>',
        font=dict(
            size=24
        )),  xaxis={'categoryorder':'total descending'})
fig.show()

#### GEOGRAPHIC LOCATION OF REVIEWERS

In [ ]:
fig = px.scatter_geo(cocoon_pharmacy_df, lat='Latitude',lon='Longitude', hover_name = 'customer_loc')
fig.update_geos(
    resolution=50,
    showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="LightBlue"
)

fig.update_layout(
    margin=dict(l=5, r=5, t=25, b=5),
    paper_bgcolor="lightGrey",
)
fig.update_traces(marker=dict(size=9, color = 'yellow', opacity = 0.6))
fig.update_layout(title = 'Customer Locations, Cocoon Pharmacy')
fig.show()

#### OVERALL PRICE VARIATION PER BRAND AND PRODUCT CATEGORY COMBINATION

In [ ]:
brand_product_prices = cocoon_pharmacy_df[['brand','product_cat','form','volume','price']].drop_duplicates()
brand_product_prices['volume'] = brand_product_prices['volume'].replace({'not available':None})
brand_product_prices['price_per_ml'] = brand_product_prices['price'].div(brand_product_prices['volume'].astype('float'))

fig = px.box(brand_product_prices, x = 'brand', y = 'price_per_ml', color= 'product_cat', points = 'all')
fig.update_xaxes(tickangle=325, title = 'Brand')
fig.update_yaxes(title = 'Price per ml')
fig.update_layout(title = 'PRICE VARIATION PER BRAND AND PRODUCT CATEGORY')
fig.update_layout(title = dict(
        text='<b>Price variation per product and brand</b>',
        font=dict(
            size=24
        )))
fig.show()


#### TEXT ANALYSIS

In [ ]:
# iterate through list of components to download from the NLTK library
for component in [
    'punkt',
    'stopwords',
    'averaged_perceptron_tagger',
    'wordnet',
    'omw-1.4'
]:
    nltk.download(
        component
    )

In [ ]:
# remove special characthers from the body of the reviews and convert all text to lowercase
cocoon_pharmacy_df['body_review'] = cocoon_pharmacy_df['body_review'].str.lower().apply(
    lambda x:  re.sub(
        r'[^\w\s]',
        '',
        x
    )
)

In [ ]:
def makewordcloud(df):
    words = ' '.join(df['body_review'])
    tokens = word_tokenize(words)
    english_stops = set(stopwords.words('english'))
    words_fin = " ".join([w for w in tokens if len(w) > 2 and w not in english_stops and w and w.isalpha()])
    lemmatizer = WordNetLemmatizer()
    word_pos = pos_tag(word_tokenize(words_fin))
    string_l = ''
    for i in range (0, len(word_pos)):
        if "V" in word_pos[i][1][0]:
            pos = 'v'
        else:
            pos = 'n'
        lem = lemmatizer.lemmatize(word_pos[i][0], pos)
        string_l += (lem) + ' '
        
    wordcloud = WordCloud(stopwords=STOPWORDS, prefer_horizontal=0.9, max_words = 100).generate(string_l)
 
    return wordcloud

In [ ]:
col = 0
fig,ax = plt.subplots(nrows = 1, ncols = 4, figsize = (25,10))
for product_cat in cocoon_pharmacy_df['product_cat'].value_counts().sort_values(ascending = False)[:4].index:
    print(product_cat)
    ax[col].imshow(makewordcloud(cocoon_pharmacy_df[cocoon_pharmacy_df['product_cat'] == product_cat]))
    ax[col].axis("off")
    ax[col].set_title('Wordcloud ' + product_cat,fontsize = 18)
    col+=1
fig.tight_layout()
plt.show()